<a href="https://colab.research.google.com/github/rishike/langchain/blob/master/langchain_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb faiss-cpu google-genai tiktoken langchain-google-genai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Wikipedia Retriever**

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [3]:
retriever

WikipediaRetriever(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.11/dist-packages/wikipedia/__init__.py'>, top_k_results=2)

In [4]:
query = "about ipl"

docs = retriever.invoke(query)

In [5]:
docs

[Document(metadata={'title': '2025 Indian Premier League', 'summary': 'The 2025 Indian Premier League, also known as IPL 18 and branded as TATA IPL 2025, is the 18th edition of the Indian Premier League. The tournament features 10 teams competing in 74 matches from 22 March to 25 May 2025. The league is being held across 13 cities in India, with Kolkata hosting the opening ceremony as well as the final since the Kolkata Knight Riders are the defending champions.\n\n', 'source': 'https://en.wikipedia.org/wiki/2025_Indian_Premier_League'}, page_content='The 2025 Indian Premier League, also known as IPL 18 and branded as TATA IPL 2025, is the 18th edition of the Indian Premier League. The tournament features 10 teams competing in 74 matches from 22 March to 25 May 2025. The league is being held across 13 cities in India, with Kolkata hosting the opening ceremony as well as the final since the Kolkata Knight Riders are the defending champions.\n\n\n== Background ==\nThe Indian Premier Leag

In [6]:
for i, doc in enumerate(docs):
  print(f"\n-- result {i+1} ---")
  print(f"Content:\n{doc.page_content}\n")
  print(f"Metadata:\n{doc.metadata}\n")


-- result 1 ---
Content:
The 2025 Indian Premier League, also known as IPL 18 and branded as TATA IPL 2025, is the 18th edition of the Indian Premier League. The tournament features 10 teams competing in 74 matches from 22 March to 25 May 2025. The league is being held across 13 cities in India, with Kolkata hosting the opening ceremony as well as the final since the Kolkata Knight Riders are the defending champions.


== Background ==
The Indian Premier League is a franchise Twenty20 cricket league held in India, organized by the Board of Control for Cricket in India (BCCI). It is held annually since the first edition in 2008. Kolkata Knight Riders are the defending champions, having won their third title in the previous season after beating Sunrisers Hyderabad in the final.


=== Format ===
Each team would play twice against the teams in their group and the team in the same row in the other group, and once against the remaining four teams in the other group. After the group stage, t

**Vector Store**

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [8]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [9]:
import os
from getpass import getpass
os.environ["GOOGLE_API_KEY"] =  getpass("Provide your Google API key here")

Provide your Google API key here··········


In [10]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [11]:
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name='my_collection'
)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [13]:
query = "What is Chroma used for ?"
results = retriever.invoke(query)

In [14]:
results

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.')]

In [15]:
for i, doc in enumerate(results):
  print(f"\n-- result {i+1} ---")
  print(f"Content:\n{doc.page_content}\n")
  print(f"Metadata:\n{doc.metadata}\n")


-- result 1 ---
Content:
Chroma is a vector database optimized for LLM-based search.

Metadata:
{}


-- result 2 ---
Content:
Embeddings convert text into high-dimensional vectors.

Metadata:
{}



**MMR**

In [16]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
from langchain_community.vectorstores import FAISS

In [18]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [20]:
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [28]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "lambda_mult": 0.1}
)

In [31]:
query = "What is langchain?"
results = retriever.invoke(query)

In [30]:
for i, doc in enumerate(results):
  print(f"\n-- result {i+1} ---")
  print(f"Content:\n{doc.page_content}\n")


-- result 1 ---
Content:
LangChain supports Chroma, FAISS, Pinecone, and more.


-- result 2 ---
Content:
MMR helps you get diverse results when doing similarity search.


-- result 3 ---
Content:
Embeddings are vector representations of text.



**Multi-Query Retriever**

In [32]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI

In [33]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [34]:
vectorstore = FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)

In [35]:
similarity_retreiver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [49]:
multiquery_retreiver = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
)

In [38]:
query = "how to improve energy levels and maintain balance?"

In [47]:
similarity_results = similarity_retreiver.invoke(query)

In [50]:
multiquery_results= multiquery_retreiver.invoke(query)

In [51]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of

**Contextual Compression Retriever**

The Contextual compression Retriever in Langchain is an advanced retriever that improves retrieval quality by compressing document after retreival - keeping only the relevant content based on the users query.

In [52]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [53]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [55]:
# Create a FAISS vector store from the documents
vectorstore = FAISS.from_documents(docs, embedding_model)

In [56]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [58]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
compressor = LLMChainExtractor.from_llm(llm)

In [59]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [60]:
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [61]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis does not occur in animal cells.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
